# Notebook de benchmarking de la Ecuación de Poisson

In [1]:
import FVM as fvm
from FVM import Mesh

## Ecuación de Poisson 3D

In [25]:
α = 100
lx = ly = lz = α/10
Tn = 100 # ºC
nvx = α
nvy = α
nvz = α
k = 1_000

### Construcción de la malla

In [26]:
%%timeit
malla = fvm.Mesh(3, volumes = (nvx, nvy, nvz), lengths = (lx, ly, lz))
malla.tag_wall_dirichlet("S", Tn)
malla.tag_wall_dirichlet(["W", "E", "T", "N", "B"], [0,0,0,0,0])

3.43 s ± 19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
times_mesh = {10 : "4.24 ms ± 193 µs", 20 : "27.5 ms ± 1.12 ms", 50 : "390 ms ± 3.94", 100 : "3.43 s ± 19 ms"}

### Haciendo cuentitas del FVM

In [27]:
%%timeit
coef = fvm.Coefficients(malla)
coef.set_diffusion(k);

2.01 ms ± 12.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
times_setting_boundary_conditions = {10 : "2 ms ± 16.2 µs", 20 : "2.19 ms ± 11 µs", 50 : "2.01 ms ± 20.6 µs", 
                                     100 : "2.01 ms ± 12.7 µs"}

### Obteniendo soluciones

In [28]:
%%timeit
sistema = fvm.EqSystem(coef)
sistema.get_solution()

1.58 ms ± 15.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
times_solutions = {10 : "1.65 ms ± 28 µs", 20 : "1.65 ms ± 48.1 µs", 50 : "1.57 ms ± 3.77 µs", 
                   100 : "1.58 ms ± 15.1 µs"}